In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 코사인 유사도 분석

In [2]:
sofaDF = pd.read_csv('./project/sofa.csv', index_col=0)
sofaDF['image'] 

0      ./static/img/living/sofa/sofa1.png
1      ./static/img/living/sofa/sofa2.png
2      ./static/img/living/sofa/sofa3.png
3      ./static/img/living/sofa/sofa4.png
4      ./static/img/living/sofa/sofa5.png
                     ...                 
94    ./static/img/living/sofa/sofa95.png
95    ./static/img/living/sofa/sofa96.png
96    ./static/img/living/sofa/sofa97.png
97    ./static/img/living/sofa/sofa98.png
98    ./static/img/living/sofa/sofa99.png
Name: image, Length: 99, dtype: object

In [4]:
# 명사 추출
from konlpy.tag import Okt

okt = Okt()

box = []
for i in tqdm(sofaDF['review']):
    a = okt.nouns(i)
    # 하나의 문장으로 합쳐지게 join 시킴
    box.append(' '.join(a))

sofaDF['nouns'] = box

100%|██████████| 99/99 [00:22<00:00,  4.34it/s]


In [5]:
sofaDF.head(1)

,brand,item,price,image,link,review,nouns
0,아이와비,카페 방수패브릭 접이식 소파베드 4colors,"119,000",./static/img/living/sofa/sofa1.png,https://ohou.se/productions/48328/selling?affe...,['푹신해서 맨날 침대 냅두고 쇼파에서 자요.. 근데 쓴지 얼마 안됐는데 벌써 꺼졋...,침대 쇼파 자 얼마 벌써 꺼졋어 서비스 주신 쿠션 팔뚝 만해 사람 고영희 룸 거실 ...


In [17]:
# 중간 저장
sofaDF.to_csv('sofaDF.csv')

In [9]:
sofaDF = pd.read_csv('./sofaDF.csv', index_col=0)
sofaDF.head(1)

,brand,item,price,image,link,review,nouns
0,아이와비,카페 방수패브릭 접이식 소파베드 4colors,"119,000",./static/img/living/sofa/sofa1.png,https://ohou.se/productions/48328/selling?affe...,['푹신해서 맨날 침대 냅두고 쇼파에서 자요.. 근데 쓴지 얼마 안됐는데 벌써 꺼졋...,침대 쇼파 자 얼마 벌써 꺼졋어 서비스 주신 쿠션 팔뚝 만해 사람 고영희 룸 거실 ...


In [10]:
# 단어 카운트 기반의 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(sofaDF['nouns']) # dtm : 텍스트를 단어의 등장 횟수로 표현된 문서-단어 행렬(document-term matrix)
dtm_sofa = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names_out()) # dtm.toarray() : 희소행렬인 dtm을 밀집행렬로 변환
dtm_sofa

,가가,가객,가갹,가게,가격,가계,가공,가관,가구,가구라,...,히히히,힌더,힌색,힐것,힐링,힐시,힘능,힘들엇어,힙니,힙부분
0,0,0,0,0,7,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,21,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,5,0,0,0,3,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,0,0,0,6,0,0,0,2,0,...,0,1,0,0,0,0,0,0,0,0
95,0,0,0,0,15,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,11,0,0,0,6,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,10,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
# dtm 파일 저장
dtm_sofa.to_csv('dtm_sofa.csv')

---

In [16]:
import pandas as pd 
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma

sofaDF = pd.read_csv('./sofaDF.csv', index_col=0)
dtm_sofa = pd.read_csv('./dtm_sofa.csv', index_col=0)

# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
# a, b : 두 벡터
def make_cosine(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

kkma = Kkma()
text = '앉아도 꺼지지 않는 폭식한 소파'
strlist = kkma.nouns(text)
text = [' '.join(strlist)]

result = cv.transform(text) # 위에서 fit 시켰기 때문에 transform만 해주면 됨
input_text = result.toarray()

brand_list = np.array(sofaDF['brand'])
item_list = np.array(sofaDF['item'])
price_list = np.array(sofaDF['price'])
image_list = np.array(sofaDF['image'])

result_cosine = make_cosine(dtm_sofa, input_text.reshape(-1))
result_args = result_cosine.argsort()[::-1] # argsort() : 배열을 정렬했을 때의 인덱스를 반환하는 함수

# 내림차순 정렬 => [result_args][0] : 코사인 유사도가 가장 큼
print(brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0])

듀커소파 DK080 4인소파/모듈형 풀커버 패브릭 소파 (1+1 커버증정) 649,900 ./static/img/living/sofa/sofa62.png


In [ ]:
# 위의 코드 함수화
def find_sofa(text):

    import pandas as pd 
    import numpy as np
    from konlpy.tag import Okt
    from konlpy.tag import Kkma

    sofaDF = pd.read_csv('./sofaDF.csv', index_col=0)
    dtm_sofa = pd.read_csv('./dtm_sofa.csv', index_col=0)

    # 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
    def make_cosine(a,b):
        return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

    kkma = Kkma()
    search_text = [text]
    strlist = kkma.nouns(search_text[0])
    search_text = [(" ".join(strlist))]

    result = cv.transform(search_text)
    search_words = result.toarray()

    brand_list = np.array(sofaDF['brand'])
    item_list = np.array(sofaDF['item'])
    price_list = np.array(sofaDF['price'])
    image_list = np.array(sofaDF['image'])

    result_cosine = make_cosine(dtm_sofa, search_words.reshape(-1))

    result_args = result_cosine.argsort()[::-1]
    return brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0]